In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_curve, auc, precision_score, f1_score, 
                           matthews_corrcoef, accuracy_score, recall_score,
                           confusion_matrix)
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import time
import os
import warnings
import gc
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast

In [ ]:
warnings.filterwarnings('ignore')

# Set font family globally
plt.rcParams['font.family'] = 'Times New Roman'
dpi = 1000
plt.rcParams['figure.dpi'] = dpi

In [ ]:
# Base directory
BASE_DIR = "AttBiLSTM_Analysis_With_FS_Without_Early_Stop"
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'models'), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'plots'), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'results'), exist_ok=True)

# Dataset sizes to compare
DATASET_SIZES = [6000, 7000, 8000, 9000]

In [ ]:
# Device setup
device = torch.device("cpu")
try:
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("MPS device detected, using MPS")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print("CUDA device detected, using CUDA")
    print(f"Using device: {device}")
except:
    print("Error detecting device capabilities, defaulting to CPU")

In [ ]:
# Load and prepare data
print("Loading and preparing data...")
try:
    data = pd.read_csv('../dataset/Combined_Common_Genes_With_Target_ML_Selected_Top_9000_Features.csv')
    print(f"Full dataset info:")
    print(f"Shape: {data.shape}")
    print(f"Target column: {data.columns[-1]}")
    
    # Separate features and target
    X_full = data.iloc[:, :-1].values  # All columns except last
    y_full = data.iloc[:, -1].values   # Last column (target)
    
    print(f"Full features shape: {X_full.shape}")
    print(f"Full target shape: {y_full.shape}")
    
    # Check target distribution
    unique_targets, counts = np.unique(y_full, return_counts=True)
    print(f"Target distribution:")
    for target, count in zip(unique_targets, counts):
        print(f"  Class {target}: {count} samples ({count/len(y_full)*100:.2f}%)")
    
    num_classes = len(unique_targets)
    num_features = X_full.shape[1]
    
except Exception as e:
    print(f"Error loading data: {e}")
    raise

In [ ]:
# Custom activation functions
class Mish(nn.Module):
    """Mish activation function: x * tanh(softplus(x))"""
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))

class Swish(nn.Module):
    """Swish activation function: x * sigmoid(x)"""
    def forward(self, x):
        return x * torch.sigmoid(x)

# Register custom activations
nn.Mish = Mish
nn.Swish = Swish

In [ ]:
# Advanced Attention Mechanism
class MultiScaleAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads=4):
        super(MultiScaleAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        
        # Multiple attention heads with different perspectives
        self.query_nets = nn.ModuleList([
            nn.Linear(hidden_dim, self.head_dim) for _ in range(num_heads)
        ])
        self.key_nets = nn.ModuleList([
            nn.Linear(hidden_dim, self.head_dim) for _ in range(num_heads)
        ])
        self.value_nets = nn.ModuleList([
            nn.Linear(hidden_dim, self.head_dim) for _ in range(num_heads)
        ])
        
        self.output_proj = nn.Linear(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(0.1)
        self.layer_norm = nn.LayerNorm(hidden_dim)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        # Multi-head attention
        attention_outputs = []
        attention_weights_list = []
        
        for i in range(self.num_heads):
            query = self.query_nets[i](x)  # [batch, seq_len, head_dim]
            key = self.key_nets[i](x)
            value = self.value_nets[i](x)
            
            # Compute attention scores
            scores = torch.matmul(query, key.transpose(-2, -1)) / np.sqrt(self.head_dim)
            attention_weights = F.softmax(scores, dim=-1)
            attention_weights_list.append(attention_weights)
            
            # Apply attention to values
            attended = torch.matmul(attention_weights, value)
            attention_outputs.append(attended)
        
        # Concatenate all heads
        multi_head_output = torch.cat(attention_outputs, dim=-1)
        
        # Project and apply residual connection
        output = self.output_proj(multi_head_output)
        output = self.dropout(output)
        output = self.layer_norm(x + output)  # Residual connection
        
        return output, attention_weights_list

In [ ]:
# Advanced AttBiLSTM Model
class AdvancedAttBiLSTM(nn.Module):
    """Advanced Attention-based Bidirectional LSTM with custom activation functions"""
    
    def __init__(self, input_dim, num_classes, hidden_dim=128, num_layers=2, dropout=0.3):
        super(AdvancedAttBiLSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_classes = num_classes
        
        # Input projection layer
        self.input_projection = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU(),  # GELU activation instead of ReLU
            nn.Dropout(dropout)
        )
        
        # Bidirectional LSTM layers
        self.lstm_layers = nn.ModuleList()
        for i in range(num_layers):
            input_size = hidden_dim if i == 0 else hidden_dim * 2
            self.lstm_layers.append(
                nn.LSTM(
                    input_size=input_size,
                    hidden_size=hidden_dim,
                    num_layers=1,
                    bidirectional=True,
                    batch_first=True,
                    dropout=dropout if i < num_layers - 1 else 0
                )
            )
        
        # Multi-scale attention mechanism
        self.attention = MultiScaleAttention(hidden_dim * 2, num_heads=4)
        
        # Feature fusion layers
        self.feature_fusion = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.Mish(),  # Mish activation function
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Swish(),  # Swish activation function
            nn.Dropout(dropout // 2)
        )
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.BatchNorm1d(hidden_dim // 4),
            nn.GELU(),
            nn.Dropout(dropout // 2),
            nn.Linear(hidden_dim // 4, num_classes)
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize model weights using Xavier/He initialization"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.LSTM):
                for name, param in module.named_parameters():
                    if 'weight_ih' in name:
                        nn.init.xavier_uniform_(param.data)
                    elif 'weight_hh' in name:
                        nn.init.orthogonal_(param.data)
                    elif 'bias' in name:
                        nn.init.zeros_(param.data)
    
    def forward(self, x):
        batch_size = x.size(0)
        
        # Project input features
        x = self.input_projection(x)  # [batch_size, hidden_dim]
        x = x.unsqueeze(1)  # [batch_size, 1, hidden_dim] - create sequence dimension
        
        # Apply LSTM layers
        for lstm in self.lstm_layers:
            x, _ = lstm(x)  # [batch_size, seq_len, hidden_dim * 2]
        
        # Apply multi-scale attention
        x_attended, attention_weights = self.attention(x)  # [batch_size, seq_len, hidden_dim * 2]
        
        # Global average pooling and max pooling
        avg_pool = torch.mean(x_attended, dim=1)  # [batch_size, hidden_dim * 2]
        max_pool, _ = torch.max(x_attended, dim=1)  # [batch_size, hidden_dim * 2]
        
        # Combine pooled features
        combined_features = avg_pool + max_pool  # Element-wise addition
        
        # Feature fusion
        fused_features = self.feature_fusion(combined_features)
        
        # Final classification
        output = self.classifier(fused_features)
        
        return output, attention_weights

In [ ]:
def prepare_dataset_by_size(X_full, y_full, dataset_size):
    """Prepare dataset for specific size"""
    print(f"\nPreparing dataset with {dataset_size} samples...")
    
    # Take first n samples
    X = X_full[:dataset_size]
    y = y_full[:dataset_size]
    
    print(f"Dataset size: {X.shape}")
    
    # Check target distribution for this subset
    unique_targets, counts = np.unique(y, return_counts=True)
    print(f"Target distribution for {dataset_size} samples:")
    for target, count in zip(unique_targets, counts):
        print(f"  Class {target}: {count} samples ({count/len(y)*100:.2f}%)")
    
    # Data preprocessing and scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Split data (70% train, 15% val, 15% test)
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_scaled, y, test_size=0.3, random_state=42, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
    )
    
    print(f"Training set: {X_train.shape[0]} samples")
    print(f"Validation set: {X_val.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")
    
    return X_train, X_val, X_test, y_train, y_val, y_test, scaler

In [ ]:
def create_data_loaders(X_train, y_train, X_val, y_val, batch_size=32):
    """Create PyTorch data loaders"""
    # Convert to tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.LongTensor(y_train)
    X_val_tensor = torch.FloatTensor(X_val)
    y_val_tensor = torch.LongTensor(y_val)
    
    # Create datasets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

In [ ]:
def train_model(model, train_loader, val_loader, y_train, epochs=200):
    """Train the AttBiLSTM model without early stopping"""
    
    model.to(device)
    
    # Calculate class weights for imbalanced data
    class_counts = np.bincount(y_train)
    class_weights = torch.FloatTensor([len(y_train) / (len(class_counts) * count) 
                                     for count in class_counts]).to(device)
    
    # Loss function with class weights
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    
    # Optimizer with different learning rates for different parts
    optimizer = optim.AdamW([
        {'params': model.input_projection.parameters(), 'lr': 1e-3},
        {'params': model.lstm_layers.parameters(), 'lr': 5e-4},
        {'params': model.attention.parameters(), 'lr': 1e-3},
        {'params': model.feature_fusion.parameters(), 'lr': 1e-3},
        {'params': model.classifier.parameters(), 'lr': 1e-3}
    ], weight_decay=1e-4)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, verbose=False
    )
    
    # Mixed precision training
    use_amp = device.type == "cuda"
    scaler = GradScaler() if use_amp else None
    
    # Training history
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': []
    }
    
    start_time = time.time()
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            optimizer.zero_grad()
            
            if use_amp:
                with autocast():
                    output, _ = model(data)
                    loss = criterion(output, target)
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                output, _ = model(data)
                loss = criterion(output, target)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
            
            train_loss += loss.item()
            pred = output.argmax(dim=1)
            train_correct += pred.eq(target).sum().item()
            train_total += target.size(0)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output, _ = model(data)
                val_loss += criterion(output, target).item()
                pred = output.argmax(dim=1)
                val_correct += pred.eq(target).sum().item()
                val_total += target.size(0)
        
        # Calculate average losses and accuracies
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_acc = 100. * train_correct / train_total
        val_acc = 100. * val_correct / val_total
        
        # Update history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        # Learning rate scheduling
        scheduler.step(avg_val_loss)
        
        if epoch % 10 == 0:
            print(f'Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, '
                  f'Val Loss: {avg_val_loss:.4f}, Train Acc: {train_acc:.2f}%, '
                  f'Val Acc: {val_acc:.2f}%')
    
    training_time = time.time() - start_time
    
    return model, history, training_time

In [ ]:
def calculate_detailed_metrics(y_true, y_pred, y_pred_proba):
    """Calculate comprehensive metrics including TP, TN, FP, FN, TPR, TNR, FPR, FNR"""
    
    # Basic metrics
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # For binary classification
    if len(np.unique(y_true)) == 2:
        tn, fp, fn, tp = cm.ravel()
        
        # Rates
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # Sensitivity/Recall
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0  # Specificity
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # Fall-out
        fnr = fn / (fn + tp) if (fn + tp) > 0 else 0  # Miss rate
        
        # AUC and ROC data
        fpr_roc, tpr_roc, _ = roc_curve(y_true, y_pred_proba[:, 1])
        auc_score = auc(fpr_roc, tpr_roc)
        roc_data = (fpr_roc, tpr_roc, auc_score)
        
        detailed_metrics = {
            'ACC': acc,
            'AUC': auc_score,
            'PRE': precision,
            'SP': tnr,  # Specificity
            'SN': tpr,  # Sensitivity
            'F1': f1,
            'MCC': mcc,
            'TPR': tpr,
            'FPR': fpr,
            'TNR': tnr,
            'FNR': fnr,
            'TP': int(tp),
            'TN': int(tn),
            'FP': int(fp),
            'FN': int(fn)
        }
        
    else:
        # For multiclass classification
        # Calculate macro-averaged rates
        tpr_list, fpr_list, tnr_list, fnr_list = [], [], [], []
        tp_total, tn_total, fp_total, fn_total = 0, 0, 0, 0
        
        auc_scores = []
        
        for i in range(len(np.unique(y_true))):
            # One-vs-Rest for each class
            y_true_binary = (y_true == i).astype(int)
            y_pred_binary = (y_pred == i).astype(int)
            
            tn_i = np.sum((y_true_binary == 0) & (y_pred_binary == 0))
            fp_i = np.sum((y_true_binary == 0) & (y_pred_binary == 1))
            fn_i = np.sum((y_true_binary == 1) & (y_pred_binary == 0))
            tp_i = np.sum((y_true_binary == 1) & (y_pred_binary == 1))
            
            tp_total += tp_i
            tn_total += tn_i
            fp_total += fp_i
            fn_total += fn_i
            
            tpr_i = tp_i / (tp_i + fn_i) if (tp_i + fn_i) > 0 else 0
            tnr_i = tn_i / (tn_i + fp_i) if (tn_i + fp_i) > 0 else 0
            fpr_i = fp_i / (fp_i + tn_i) if (fp_i + tn_i) > 0 else 0
            fnr_i = fn_i / (fn_i + tp_i) if (fn_i + tp_i) > 0 else 0
            
            tpr_list.append(tpr_i)
            tnr_list.append(tnr_i)
            fpr_list.append(fpr_i)
            fnr_list.append(fnr_i)
            
            # ROC curve for each class
            fpr_roc, tpr_roc, _ = roc_curve(y_true_binary, y_pred_proba[:, i])
            auc_i = auc(fpr_roc, tpr_roc)
            auc_scores.append(auc_i)
        
        # For multiclass, create averaged ROC curve
        fpr_avg = np.linspace(0, 1, 100)
        tpr_avg = np.zeros_like(fpr_avg)
        for i in range(len(np.unique(y_true))):
            y_true_binary = (y_true == i).astype(int)
            fpr_i, tpr_i, _ = roc_curve(y_true_binary, y_pred_proba[:, i])
            tpr_interp = np.interp(fpr_avg, fpr_i, tpr_i)
            tpr_avg += tpr_interp
        tpr_avg /= len(np.unique(y_true))
        auc_avg = auc(fpr_avg, tpr_avg)
        roc_data = (fpr_avg, tpr_avg, auc_avg)
        
        detailed_metrics = {
            'ACC': acc,
            'AUC': np.mean(auc_scores),
            'PRE': precision,
            'SP': np.mean(tnr_list),  # Specificity
            'SN': np.mean(tpr_list),  # Sensitivity
            'F1': f1,
            'MCC': mcc,
            'TPR': np.mean(tpr_list),
            'FPR': np.mean(fpr_list),
            'TNR': np.mean(tnr_list),
            'FNR': np.mean(fnr_list),
            'TP': int(tp_total),
            'TN': int(tn_total),
            'FP': int(fp_total),
            'FN': int(fn_total)
        }
    
    return detailed_metrics, roc_data

In [ ]:
def plot_combined_roc_curves(all_test_data):
    """Plot combined ROC curves for all dataset sizes"""
    
    plt.figure(figsize=(10, 8))
    colors = ['blue', 'red', 'green', 'purple']
    
    for i, (size, roc_data) in enumerate(all_test_data.items()):
        if roc_data is not None:
            fpr, tpr, auc_score = roc_data
            plt.plot(fpr, tpr, color=colors[i], linewidth=3, 
                    label=f'{size} samples (AUC = {auc_score:.4f})')
    
    # Diagonal line
    plt.plot([0, 1], [0, 1], 'k--', linewidth=2, alpha=0.7)
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=24)
    plt.ylabel('True Positive Rate', fontsize=24)
    plt.title('ROC Curves Comparison Across Dataset Sizes', fontsize=28, pad=20)
    plt.legend(loc="lower right", fontsize=20)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.grid(False)
    
    # Enhance aesthetics
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(2)
        spine.set_color('black')
    
    plt.tight_layout()
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'combined_roc_curves.png'), dpi=dpi, bbox_inches='tight')
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'combined_roc_curves.pdf'), dpi=dpi, bbox_inches='tight')
    plt.close()

In [ ]:
def plot_separate_training_validation_loss(all_histories):
    """Plot separate training and validation loss curves"""
    
    colors = ['blue', 'red', 'green', 'purple']
    
    # 1. Training Loss Comparison
    plt.figure(figsize=(10, 8))
    for i, size in enumerate(DATASET_SIZES):
        plt.plot(all_histories[size]['train_loss'], color=colors[i], 
                linewidth=2, label=f'{size} samples')
    plt.title('Training Loss Comparison of AttBiLSTM With FS', fontsize=28, pad=20)
    plt.xlabel('Epoch', fontsize=24)
    plt.ylabel('Loss', fontsize=24)
    plt.legend(fontsize=20)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.grid(False)
    
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(2)
        spine.set_color('black')
    
    plt.tight_layout()
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'training_loss_comparison.png'), dpi=dpi, bbox_inches='tight')
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'training_loss_comparison.pdf'), dpi=dpi, bbox_inches='tight')
    plt.close()
    
    # 2. Validation Loss Comparison
    plt.figure(figsize=(10, 8))
    for i, size in enumerate(DATASET_SIZES):
        plt.plot(all_histories[size]['val_loss'], color=colors[i], 
                linewidth=2, label=f'{size} samples')
    plt.title('Validation Loss Comparison of AttBiLSTM With FS', fontsize=28, pad=20)
    plt.xlabel('Epoch', fontsize=24)
    plt.ylabel('Loss', fontsize=24)
    plt.legend(fontsize=20)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.grid(False)
    
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(2)
        spine.set_color('black')
    
    plt.tight_layout()
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'validation_loss_comparison.png'), dpi=dpi, bbox_inches='tight')
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'validation_loss_comparison.pdf'), dpi=dpi, bbox_inches='tight')
    plt.close()

In [ ]:
def plot_performance_metrics_variants(all_metrics):
    """Create different types of performance comparison graphs"""
    
    metrics_to_plot = ['ACC', 'AUC', 'PRE', 'SN', 'SP', 'F1', 'MCC']
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
    
    # 1. Classic Bar Chart
    plt.figure(figsize=(15, 10))
    x_pos = np.arange(len(DATASET_SIZES))
    
    for i, metric in enumerate(metrics_to_plot):
        plt.subplot(2, 4, i+1)
        values = [all_metrics[size][metric] for size in DATASET_SIZES]
        bars = plt.bar(x_pos, values, color=colors[i], alpha=0.7, edgecolor='black', linewidth=1)
        
        # Add value labels on bars
        for j, bar in enumerate(bars):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                    f'{height:.3f}', ha='center', va='bottom', fontsize=20)
        
        plt.title(f'{metric}', fontsize=28, fontweight='bold')
        plt.xlabel('Dataset Size', fontsize=24)
        plt.ylabel(metric, fontsize=24)
        plt.xticks(x_pos, DATASET_SIZES, fontsize=18)
        plt.yticks(fontsize=18)
        plt.ylim(0, max(values) * 1.1)
        plt.grid(False)
        
        # Enhance aesthetics
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_linewidth(1.5)
            spine.set_color('black')
    
    # Remove empty subplot
    plt.subplot(2, 4, 8)
    plt.axis('off')
    
    plt.suptitle('Performance Metrics Comparison (Bar Chart)', fontsize=32, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'metrics_comparison_bar.png'), dpi=dpi, bbox_inches='tight')
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'metrics_comparison_bar.pdf'), dpi=dpi, bbox_inches='tight')
    plt.close()
    
    # 2. Line Graph with Markers
    plt.figure(figsize=(15, 10))
    for i, metric in enumerate(metrics_to_plot):
        plt.subplot(2, 4, i+1)
        values = [all_metrics[size][metric] for size in DATASET_SIZES]
        plt.plot(DATASET_SIZES, values, 'o-', linewidth=3, markersize=8, 
                color=colors[i], markerfacecolor='white', markeredgewidth=2)
        
        # Add value annotations
        for j, (x, y) in enumerate(zip(DATASET_SIZES, values)):
            plt.annotate(f'{y:.3f}', (x, y), textcoords="offset points", 
                        xytext=(0,10), ha='center', fontsize=18)
        
        plt.title(f'{metric}', fontsize=28, fontweight='bold')
        plt.xlabel('Dataset Size', fontsize=24)
        plt.ylabel(metric, fontsize=24)
        plt.xticks(DATASET_SIZES, fontsize=18)
        plt.yticks(fontsize=18)
        plt.grid(False)
        
        # Enhance aesthetics
        ax = plt.gca()
        for spine in ax.spines.values():
            spine.set_linewidth(1.5)
            spine.set_color('black')
    
    plt.subplot(2, 4, 8)
    plt.axis('off')
    
    plt.suptitle('Performance Metrics Comparison (Line Graph)', fontsize=32, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'metrics_comparison_line.png'), dpi=dpi, bbox_inches='tight')
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'metrics_comparison_line.pdf'), dpi=dpi, bbox_inches='tight')
    plt.close()
    
    # 3. Violin Plot
    fig, axes = plt.subplots(2, 4, figsize=(20, 12))
    axes = axes.flatten()
    
    for i, metric in enumerate(metrics_to_plot):
        # Create data for violin plot (simulate some distribution around the actual values)
        violin_data = []
        for size in DATASET_SIZES:
            base_value = all_metrics[size][metric]
            # Add some synthetic variation for visualization
            variation = np.random.normal(base_value, base_value * 0.01, 50)
            violin_data.append(variation)
        
        parts = axes[i].violinplot(violin_data, positions=range(len(DATASET_SIZES)), 
                                   showmeans=True, showmedians=True)
        
        # Color the violin plots
        for pc in parts['bodies']:
            pc.set_facecolor(colors[i])
            pc.set_alpha(0.7)
        
        axes[i].set_title(f'{metric}', fontsize=28, fontweight='bold')
        axes[i].set_xlabel('Dataset Size', fontsize=24)
        axes[i].set_ylabel(metric, fontsize=24)
        axes[i].set_xticks(range(len(DATASET_SIZES)))
        axes[i].set_xticklabels(DATASET_SIZES, fontsize=18)
        axes[i].tick_params(axis='y', labelsize=18)
        axes[i].grid(False)
        
        # Enhance aesthetics
        for spine in axes[i].spines.values():
            spine.set_linewidth(1.5)
            spine.set_color('black')
    
    # Remove empty subplot
    axes[7].axis('off')
    
    plt.suptitle('Performance Metrics Comparison (Violin Plot)', fontsize=32, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'metrics_comparison_violin.png'), dpi=dpi, bbox_inches='tight')
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'metrics_comparison_violin.pdf'), dpi=dpi, bbox_inches='tight')
    plt.close()
    
    # 4. Radar Chart
    fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(projection='polar'))
    
    # Number of metrics
    N = len(metrics_to_plot)
    
    # Compute angle for each metric
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Complete the circle
    
    # Plot for each dataset size
    for i, size in enumerate(DATASET_SIZES):
        values = [all_metrics[size][metric] for metric in metrics_to_plot]
        values += values[:1]  # Complete the circle
        
        ax.plot(angles, values, 'o-', linewidth=3, label=f'{size} samples', 
                color=colors[i % len(colors)], markersize=8)
        ax.fill(angles, values, alpha=0.25, color=colors[i % len(colors)])
    
    # Add labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(metrics_to_plot, fontsize=20)
    ax.set_ylim(0, 1)
    ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=16)
    ax.grid(True)
    
    plt.title('Performance Metrics Comparison (Radar Chart)', 
              fontsize=28, fontweight='bold', pad=30)
    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize=18)
    plt.tight_layout()
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'metrics_comparison_radar.png'), dpi=dpi, bbox_inches='tight')
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'metrics_comparison_radar.pdf'), dpi=dpi, bbox_inches='tight')
    plt.close()

In [ ]:
def plot_horizontal_time_comparison(all_metrics):
    """Plot horizontal bar charts for training and testing time"""
    
    # 1. Training Time Horizontal Bar
    plt.figure(figsize=(10, 8))
    training_times = [all_metrics[size]['Training_Time'] for size in DATASET_SIZES]
    y_pos = np.arange(len(DATASET_SIZES))
    
    bars = plt.barh(y_pos, training_times, color='skyblue', alpha=0.7, 
                    edgecolor='black', linewidth=2)
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width + max(training_times)*0.01, bar.get_y() + bar.get_height()/2,
                f'{width:.1f}s', ha='left', va='center', fontsize=20)
    
    plt.title('Training Time Comparison of AttBiLSTM With FS', fontsize=28, pad=20)
    plt.xlabel('Training Time (seconds)', fontsize=24)
    plt.ylabel('Dataset Size', fontsize=24)
    plt.yticks(y_pos, [f'{size} samples' for size in DATASET_SIZES], fontsize=18)
    plt.xticks(fontsize=18)
    plt.grid(False)
    
    # Enhance aesthetics
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(2)
        spine.set_color('black')
    
    plt.tight_layout()
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'training_time_horizontal.png'), dpi=dpi, bbox_inches='tight')
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'training_time_horizontal.pdf'), dpi=dpi, bbox_inches='tight')
    plt.close()
    
    # 2. Testing Time Horizontal Bar
    plt.figure(figsize=(10, 8))
    testing_times = [all_metrics[size]['Testing_Time'] for size in DATASET_SIZES]
    
    bars = plt.barh(y_pos, testing_times, color='lightcoral', alpha=0.7, 
                    edgecolor='black', linewidth=2)
    
    # Add value labels
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width + max(testing_times)*0.01, bar.get_y() + bar.get_height()/2,
                f'{width:.3f}s', ha='left', va='center', fontsize=20)
    
    plt.title('Testing Time Comparison of AttBiLSTM With FS', fontsize=28, pad=20)
    plt.xlabel('Testing Time (seconds)', fontsize=24)
    plt.ylabel('Dataset Size', fontsize=24)
    plt.yticks(y_pos, [f'{size} samples' for size in DATASET_SIZES], fontsize=18)
    plt.xticks(fontsize=18)
    plt.grid(False)
    
    # Enhance aesthetics
    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(2)
        spine.set_color('black')
    
    plt.tight_layout()
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'testing_time_horizontal.png'), dpi=dpi, bbox_inches='tight')
    plt.savefig(os.path.join(BASE_DIR, 'plots', 'testing_time_horizontal.pdf'), dpi=dpi, bbox_inches='tight')
    plt.close()

In [ ]:
def plot_comparative_results(all_metrics, all_histories, all_test_data):
    """Create comprehensive comparative plots for all dataset sizes"""
    
    print("\nGenerating comprehensive comparative plots...")
    
    # 1. Combined ROC Curves
    plot_combined_roc_curves(all_test_data)
    print("✓ Combined ROC curves saved")
    
    # 2. Separate Training and Validation Loss
    plot_separate_training_validation_loss(all_histories)
    print("✓ Separate training/validation loss curves saved")
    
    # 3. Performance Metrics Variants
    plot_performance_metrics_variants(all_metrics)
    print("✓ Performance metrics variants (bar, line, violin, radar) saved")
    
    # 4. Horizontal Time Comparison
    plot_horizontal_time_comparison(all_metrics)
    print("✓ Horizontal time comparison charts saved")
    
    print("All comparative plots saved successfully!")

In [ ]:
def run_analysis_for_dataset_size(dataset_size):
    """Run complete analysis for a specific dataset size"""
    
    print(f"\n{'='*60}")
    print(f"RUNNING ANALYSIS FOR {dataset_size} SAMPLES")
    print(f"{'='*60}")
    
    # Prepare dataset
    X_train, X_val, X_test, y_train, y_val, y_test, scaler = prepare_dataset_by_size(
        X_full, y_full, dataset_size
    )
    
    # Create data loaders
    train_loader, val_loader = create_data_loaders(X_train, y_train, X_val, y_val, batch_size=32)
    
    # Initialize model
    model = AdvancedAttBiLSTM(
        input_dim=num_features,
        num_classes=num_classes,
        hidden_dim=128,
        num_layers=2,
        dropout=0.3
    )
    
    print(f"Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")
    
    # Train model
    print("Training model...")
    trained_model, history, training_time = train_model(
        model, train_loader, val_loader, y_train, epochs=200
    )
    
    # Save model directly to main models folder
    model_save_path = os.path.join(BASE_DIR, 'models', f'attbilstm_model_{dataset_size}.pth')
    torch.save({
        'model_state_dict': trained_model.state_dict(),
        'model_config': {
            'input_dim': num_features,
            'num_classes': num_classes,
            'hidden_dim': 128,
            'num_layers': 2,
            'dropout': 0.3
        },
        'training_history': history,
        'dataset_size': dataset_size,
        'scaler_params': {
            'mean_': scaler.mean_.tolist(),
            'scale_': scaler.scale_.tolist()
        }
    }, model_save_path)
    print(f"Model saved: {model_save_path}")
    
    # Evaluate on test set
    print("Evaluating on test set...")
    trained_model.eval()
    X_test_tensor = torch.FloatTensor(X_test).to(device)
    
    start_time = time.time()
    with torch.no_grad():
        test_outputs, attention_weights = trained_model(X_test_tensor)
        test_probabilities = F.softmax(test_outputs, dim=1).cpu().numpy()
        test_predictions = np.argmax(test_probabilities, axis=1)
    testing_time = time.time() - start_time
    
    # Calculate detailed metrics and get ROC data
    detailed_metrics, roc_data = calculate_detailed_metrics(y_test, test_predictions, test_probabilities)
    
    # Add additional information
    detailed_metrics['Dataset_Size'] = dataset_size
    detailed_metrics['Training_Time'] = training_time
    detailed_metrics['Testing_Time'] = testing_time
    detailed_metrics['Train_Samples'] = len(X_train)
    detailed_metrics['Val_Samples'] = len(X_val)
    detailed_metrics['Test_Samples'] = len(X_test)
    
    # Save results directly to main results folder
    results_df = pd.DataFrame([detailed_metrics])
    results_path = os.path.join(BASE_DIR, 'results', f'metrics_{dataset_size}.csv')
    results_df.to_csv(results_path, index=False)
    print(f"Results saved: {results_path}")
    
    # Save training history directly to main results folder
    history_df = pd.DataFrame(history)
    history_path = os.path.join(BASE_DIR, 'results', f'training_history_{dataset_size}.csv')
    history_df.to_csv(history_path, index=False)
    print(f"History saved: {history_path}")
    
    print(f"Analysis completed for {dataset_size} samples")
    print(f"Best metrics: ACC={detailed_metrics['ACC']:.4f}, AUC={detailed_metrics['AUC']:.4f}, F1={detailed_metrics['F1']:.4f}")
    
    return detailed_metrics, history, roc_data

In [ ]:
def create_comprehensive_summary(all_metrics):
    """Create comprehensive summary report"""
    
    print("\nGenerating comprehensive summary...")
    
    # Create summary DataFrame
    summary_data = []
    for size in DATASET_SIZES:
        metrics = all_metrics[size]
        summary_data.append({
            'Dataset_Size': size,
            'Train_Samples': metrics['Train_Samples'],
            'Val_Samples': metrics['Val_Samples'],
            'Test_Samples': metrics['Test_Samples'],
            'ACC': metrics['ACC'],
            'AUC': metrics['AUC'],
            'PRE': metrics['PRE'],
            'SN': metrics['SN'],
            'SP': metrics['SP'],
            'F1': metrics['F1'],
            'MCC': metrics['MCC'],
            'Training_Time': metrics['Training_Time'],
            'Testing_Time': metrics['Testing_Time']
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Save summary
    summary_path = os.path.join(BASE_DIR, 'results', 'comprehensive_summary.csv')
    summary_df.to_csv(summary_path, index=False)
    
    # Create detailed text report
    report_path = os.path.join(BASE_DIR, 'results', 'analysis_report.txt')
    with open(report_path, 'w') as f:
        f.write("AttBiLSTM Dataset Size Comparison Analysis Report (WITH FEATURE SELECTION)\n")
        f.write("=" * 70 + "\n\n")
        
        f.write("EXECUTIVE SUMMARY\n")
        f.write("-" * 20 + "\n")
        f.write(f"This analysis compares AttBiLSTM model performance across ")
        f.write(f"different dataset sizes: {', '.join(map(str, DATASET_SIZES))} samples.\n")
        f.write(f"Dataset used: Combined_Common_Genes_With_Target_ML_Selected_Top_9000_Features.csv\n\n")
        
        # Find best performing size for each metric
        best_performers = {}
        for metric in ['ACC', 'AUC', 'F1', 'MCC']:
            best_size = max(DATASET_SIZES, key=lambda x: all_metrics[x][metric])
            best_value = all_metrics[best_size][metric]
            best_performers[metric] = (best_size, best_value)
        
        f.write("BEST PERFORMING DATASET SIZES:\n")
        for metric, (size, value) in best_performers.items():
            f.write(f"  {metric}: {size} samples ({value:.4f})\n")
        f.write("\n")
        
        # Performance trends
        f.write("PERFORMANCE TRENDS:\n")
        f.write("-" * 20 + "\n")
        for metric in ['ACC', 'AUC', 'F1', 'MCC']:
            values = [all_metrics[size][metric] for size in DATASET_SIZES]
            trend = "Increasing" if values[-1] > values[0] else "Decreasing"
            improvement = ((values[-1] - values[0]) / values[0]) * 100
            f.write(f"{metric}: {trend} trend ({improvement:+.2f}% change from 6K to 9K samples)\n")
        f.write("\n")
        
        # Training efficiency
        f.write("TRAINING EFFICIENCY:\n")
        f.write("-" * 20 + "\n")
        training_times = [all_metrics[size]['Training_Time'] for size in DATASET_SIZES]
        f.write(f"Training time range: {min(training_times):.1f}s - {max(training_times):.1f}s\n")
        f.write(f"Time increase from 6K to 9K: {((training_times[-1] - training_times[0]) / training_times[0]) * 100:.1f}%\n\n")
        
        # Detailed metrics table
        f.write("DETAILED METRICS TABLE:\n")
        f.write("-" * 25 + "\n")
        f.write(summary_df.round(4).to_string(index=False))
        f.write("\n\n")
        
        # Recommendations
        f.write("RECOMMENDATIONS:\n")
        f.write("-" * 15 + "\n")
        
        # Find most balanced performer (considering accuracy, F1, and training time)
        balanced_scores = []
        for size in DATASET_SIZES:
            metrics = all_metrics[size]
            # Normalize metrics (higher is better for performance, lower is better for time)
            norm_acc = metrics['ACC']
            norm_f1 = metrics['F1']
            norm_time = 1 / (metrics['Training_Time'] / min(training_times))  # Inverse for time
            balanced_score = (norm_acc + norm_f1 + norm_time) / 3
            balanced_scores.append((size, balanced_score))
        
        best_balanced = max(balanced_scores, key=lambda x: x[1])
        f.write(f"1. Most balanced performance: {best_balanced[0]} samples\n")
        f.write(f"   (Considering accuracy, F1-score, and training efficiency)\n\n")
        
        f.write(f"2. For maximum accuracy: {best_performers['ACC'][0]} samples ")
        f.write(f"({best_performers['ACC'][1]:.4f})\n\n")
        
        f.write(f"3. For best F1-score: {best_performers['F1'][0]} samples ")
        f.write(f"({best_performers['F1'][1]:.4f})\n\n")
        
        # Sample efficiency analysis
        sample_efficiency = []
        for i, size in enumerate(DATASET_SIZES):
            if i > 0:
                prev_size = DATASET_SIZES[i-1]
                acc_gain = all_metrics[size]['ACC'] - all_metrics[prev_size]['ACC']
                sample_gain = size - prev_size
                efficiency = acc_gain / sample_gain * 1000  # Per 1000 samples
                sample_efficiency.append((size, efficiency))
        
        f.write("SAMPLE EFFICIENCY ANALYSIS:\n")
        f.write("-" * 30 + "\n")
        f.write("Accuracy gain per 1000 additional samples:\n")
        for size, efficiency in sample_efficiency:
            f.write(f"  Up to {size} samples: {efficiency:.6f}\n")
        
        if sample_efficiency:
            best_efficiency = max(sample_efficiency, key=lambda x: x[1])
            f.write(f"\nMost efficient range: Up to {best_efficiency[0]} samples\n")
    
    print(f"Comprehensive summary saved to: {summary_path}")
    print(f"Detailed report saved to: {report_path}")
    
    return summary_df

In [ ]:
def run_complete_dataset_size_comparison():
    """Run the complete dataset size comparison analysis"""
    
    print("=" * 70)
    print("ADVANCED ATTBILSTM DATASET SIZE COMPARISON ANALYSIS (WITH FS)")
    print("=" * 70)
    print(f"Comparing performance across dataset sizes: {DATASET_SIZES}")
    print(f"Full dataset size: {len(X_full)} samples")
    print(f"Number of features: {num_features}")
    print(f"Number of classes: {num_classes}")
    
    # Store results for all dataset sizes
    all_metrics = {}
    all_histories = {}
    all_test_data = {}
    
    # Run analysis for each dataset size
    for dataset_size in DATASET_SIZES:
        try:
            metrics, history, roc_data = run_analysis_for_dataset_size(dataset_size)
            all_metrics[dataset_size] = metrics
            all_histories[dataset_size] = history
            all_test_data[dataset_size] = roc_data
            
            # Memory cleanup after each run
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            elif torch.backends.mps.is_available():
                torch.mps.empty_cache()
            gc.collect()
            
        except Exception as e:
            print(f"Error analyzing dataset size {dataset_size}: {e}")
            continue
    
    # Generate comparative analysis
    if len(all_metrics) > 1:
        print(f"\n{'='*60}")
        print("GENERATING COMPARATIVE ANALYSIS")
        print(f"{'='*60}")
        
        # Create comprehensive comparative plots
        plot_comparative_results(all_metrics, all_histories, all_test_data)
        
        # Create comprehensive summary
        summary_df = create_comprehensive_summary(all_metrics)
        
        # Print final comparison
        print("\n" + "="*60)
        print("FINAL COMPARISON SUMMARY")
        print("="*60)
        
        comparison_metrics = ['ACC', 'AUC', 'F1', 'MCC', 'Training_Time']
        print(f"{'Size':<8}", end="")
        for metric in comparison_metrics:
            print(f"{metric:<12}", end="")
        print()
        print("-" * 60)
        
        for size in DATASET_SIZES:
            if size in all_metrics:
                metrics = all_metrics[size]
                print(f"{size:<8}", end="")
                for metric in comparison_metrics:
                    if metric == 'Training_Time':
                        print(f"{metrics[metric]:<12.1f}", end="")
                    else:
                        print(f"{metrics[metric]:<12.4f}", end="")
                print()
        
        # Find overall best performer
        print(f"\n{'='*60}")
        print("KEY FINDINGS")
        print("="*60)
        
        best_acc = max(DATASET_SIZES, key=lambda x: all_metrics.get(x, {}).get('ACC', 0))
        best_f1 = max(DATASET_SIZES, key=lambda x: all_metrics.get(x, {}).get('F1', 0))
        fastest = min(DATASET_SIZES, key=lambda x: all_metrics.get(x, {}).get('Training_Time', float('inf')))
        
        print(f"🏆 Best Accuracy: {best_acc} samples ({all_metrics[best_acc]['ACC']:.4f})")
        print(f"🎯 Best F1-Score: {best_f1} samples ({all_metrics[best_f1]['F1']:.4f})")
        print(f"⚡ Fastest Training: {fastest} samples ({all_metrics[fastest]['Training_Time']:.1f}s)")
        
        # Performance improvement analysis
        first_size = min(DATASET_SIZES)
        last_size = max(DATASET_SIZES)
        if first_size in all_metrics and last_size in all_metrics:
            acc_improvement = ((all_metrics[last_size]['ACC'] - all_metrics[first_size]['ACC']) / 
                             all_metrics[first_size]['ACC']) * 100
            time_increase = ((all_metrics[last_size]['Training_Time'] - all_metrics[first_size]['Training_Time']) / 
                           all_metrics[first_size]['Training_Time']) * 100
            
            print(f"\n📈 Performance change from {first_size} to {last_size} samples:")
            print(f"   Accuracy: {acc_improvement:+.2f}%")
            print(f"   Training time: {time_increase:+.1f}%")
    
    print(f"\n{'='*70}")
    print("ANALYSIS COMPLETED SUCCESSFULLY!")
    print("="*70)
    print(f"All results saved in directory: {BASE_DIR}")
    print("\nGenerated files:")
    print("📁 Models saved in: AttBiLSTM_Analysis_With_FS/models/")
    print("   - attbilstm_model_6000.pth")
    print("   - attbilstm_model_7000.pth") 
    print("   - attbilstm_model_8000.pth")
    print("   - attbilstm_model_9000.pth")
    print("📊 All plots saved in: AttBiLSTM_Analysis_With_FS/plots/")
    print("   - combined_roc_curves.png/pdf")
    print("   - training_loss_comparison.png/pdf")
    print("   - validation_loss_comparison.png/pdf")
    print("   - metrics_comparison_bar.png/pdf")
    print("   - metrics_comparison_line.png/pdf")
    print("   - metrics_comparison_violin.png/pdf")
    print("   - metrics_comparison_radar.png/pdf")
    print("   - training_time_horizontal.png/pdf")
    print("   - testing_time_horizontal.png/pdf")
    print("📋 All results saved in: AttBiLSTM_Analysis_With_FS/results/")
    print("   - metrics_XXXX.csv (for each dataset size)")
    print("   - training_history_XXXX.csv (for each dataset size)")
    print("   - comprehensive_summary.csv")
    print("   - analysis_report.txt")
    
    return all_metrics, all_histories

In [ ]:
if __name__ == "__main__":
    try:
        all_metrics, all_histories = run_complete_dataset_size_comparison()
        
        # Final memory cleanup
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        elif torch.backends.mps.is_available():
            torch.mps.empty_cache()
        gc.collect()
        
        print("\n✅ Dataset size comparison analysis completed successfully!")
        
    except Exception as e:
        print(f"❌ Error during analysis: {e}")
        import traceback
        traceback.print_exc()
        
    finally:
        print("🏁 Analysis pipeline finished.")